# Chapter 4 — Determinants (Colab Companion)

This notebook provides **Python helpers and demos** for core Chapter 4 topics on determinants:

- Definitions: **Leibniz (permutation) formula** and **Laplace (cofactor) expansion**
- **Minors**, **cofactors**, **adjugate**, and the formula `A^{-1} = adj(A)/det(A)` when invertible
- Effect of **elementary row operations** on `det(A)`
- Properties: `det(AB) = det(A) det(B)`, `det(A^T) = det(A)`, determinant of **triangular** matrices
- Determinant via **LU decomposition** (with permutations) and via **product of eigenvalues**
- **Cramer's rule**
- **Geometric meaning**: area/volume as `|det|` (2D/3D visual demos)

All exact symbolic computations use **SymPy**; small educational re-implementations are included for transparency.

In [ ]:
# Setup (NumPy/SymPy/itertools/matplotlib). In Colab these are typically available.
# If needed, you can uncomment the next line to install/upgrade:
# !pip -q install numpy sympy matplotlib

import numpy as np
import sympy as sp
from itertools import permutations
import math

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

np.set_printoptions(suppress=True, linewidth=120)
sp.init_printing(use_unicode=True)

def to_M(A):
    """Coerce to a SymPy Matrix with rational entries when feasible."""
    if isinstance(A, sp.MatrixBase):
        M = A.copy()
    else:
        M = sp.Matrix(A)
    def _coerce(x):
        if isinstance(x, (int, np.integer)):
            return sp.Rational(int(x))
        try:
            return sp.nsimplify(x)
        except Exception:
            return x
    return M.applyfunc(_coerce)

## 1) Determinant via Leibniz (permutation) formula

For small `n`, we can compute `det(A)` directly from permutations to see what SymPy does internally.
This is **O(n!)**, so it's just for demonstration.

In [ ]:
def det_leibniz(A):
    """Compute det(A) by the Leibniz formula (slow; for small n only)."""
    M = to_M(A)
    n = M.shape[0]
    assert n == M.shape[1], "Matrix must be square."
    s = sp.Integer(0)
    for p in permutations(range(n)):
        # sign of permutation p
        sign = sp.Integer(1)
        # compute parity by counting inversions
        inv = 0
        for i in range(n):
            for j in range(i+1, n):
                if p[i] > p[j]:
                    inv += 1
        if inv % 2 == 1:
            sign = -sign
        # product of entries along permutation
        prod = sp.Integer(1)
        for i in range(n):
            prod *= M[i, p[i]]
        s += sign * prod
    return sp.simplify(s)

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[1,2,3],[4,0,6],[7,8,9]])
    print("det by Leibniz:", det_leibniz(A))
    print("det by SymPy:  ", A.det())

## 2) Minors, cofactors, and Laplace (cofactor) expansion

We implement minors/cofactors and expansion along a chosen row/column with a step-by-step log.

In [ ]:
def minor(M, i, j):
    """Minor M_ij: determinant of M with row i and column j removed (0-based indices)."""
    M = to_M(M)
    return M.minor_submatrix(i, j).det()

def cofactor(M, i, j):
    """Cofactor C_ij = (-1)^{i+j} * minor(M, i, j)."""
    return (-1)**(i+j) * minor(M, i, j)

def laplace_expand_row(M, i):
    """Laplace expansion along row i (0-based). Returns (det, steps)."""
    M = to_M(M)
    assert M.shape[0] == M.shape[1], "Square matrix required."
    n = M.shape[0]
    steps = []
    total = sp.Integer(0)
    for j in range(n):
        aij = M[i, j]
        Cij = cofactor(M, i, j)
        term = aij * Cij
        steps.append((i, j, aij, Cij, term))
        total += term
    return sp.simplify(total), steps

def laplace_expand_col(M, j):
    """Laplace expansion along column j (0-based). Returns (det, steps)."""
    M = to_M(M)
    assert M.shape[0] == M.shape[1], "Square matrix required."
    n = M.shape[0]
    steps = []
    total = sp.Integer(0)
    for i in range(n):
        aij = M[i, j]
        Cij = cofactor(M, i, j)
        term = aij * Cij
        steps.append((i, j, aij, Cij, term))
        total += term
    return sp.simplify(total), steps

# Demo
if __name__ == "__main__":
    M = sp.Matrix([[2,1,3],[0,1,-1],[4,2,2]])
    d_row2, steps = laplace_expand_row(M, 1)
    d_col1, _ = laplace_expand_col(M, 0)
    print("Laplace along row 2:", d_row2)
    print("Laplace along col 1:", d_col1)
    print("SymPy det:", M.det())

## 3) Cofactor matrix, adjugate, and inverse formula

`adj(A) = C^T`, where `C` is the cofactor matrix. If `det(A) ≠ 0` then `A^{-1} = adj(A)/det(A)`.
We also verify `A · adj(A) = det(A) · I`.

In [ ]:
def cofactor_matrix(M):
    M = to_M(M)
    n = M.shape[0]
    assert n == M.shape[1], "Square matrix required."
    C = sp.zeros(n, n)
    for i in range(n):
        for j in range(n):
            C[i, j] = cofactor(M, i, j)
    return C

def adjugate(M):
    return cofactor_matrix(M).T

def inverse_via_adjugate(M):
    M = to_M(M)
    d = M.det()
    if d == 0:
        raise ValueError("Matrix is singular; adjugate formula requires det != 0.")
    return adjugate(M) / d

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[1,2,1],[0,1,3],[2,0,1]])
    adjA = adjugate(A)
    print("A * adj(A) == det(A)*I ? ", (A*adjA == A.det()*sp.eye(A.shape[0])))
    print("A^{-1} via adjugate == SymPy inv? ", inverse_via_adjugate(A).equals(A.inv()))

## 4) Row operations and key determinant properties

- Swap two rows → `det` changes sign.  
- Multiply a row by `k` → `det` scales by `k`.  
- Add a multiple of one row to another → `det` unchanged.  
- `det(AB) = det(A) det(B)`, `det(A^T) = det(A)`.  
- Triangular matrices: `det` is the product of diagonal entries.

In [ ]:
def det_after_row_swap(A, i, j):
    A = to_M(A)
    B = A.copy(); B.row_swap(i, j)
    return A.det(), B.det()

def det_after_row_scale(A, i, k):
    A = to_M(A)
    B = A.copy(); B[i,:] *= to_M([[k]])[0,0]
    return A.det(), B.det()

def det_after_row_add(A, i, j, k):
    A = to_M(A)
    B = A.copy(); B[i,:] += to_M([[k]])[0,0] * B[j,:]
    return A.det(), B.det()

def check_multiplicativity(A, B):
    A, B = to_M(A), to_M(B)
    return (A*B).det() == A.det() * B.det()

def check_transpose_invariance(A):
    A = to_M(A)
    return A.T.det() == A.det()

def det_triangular(diag_entries):
    D = sp.diag(*diag_entries)
    return D.det(), D

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[2,1,0],[0,3,4],[5,0,1]])
    print("Swap rows 0 and 2: det signs:", det_after_row_swap(A,0,2))
    print("Scale row 1 by -2:", det_after_row_scale(A,1,-2))
    print("R2 := R2 + 3 R1:", det_after_row_add(A,1,0,3))
    B = sp.Matrix([[1,2,3],[0,4,5],[1,0,6]])
    print("det(AB) = det(A)det(B)?", check_multiplicativity(A,B))
    print("det(A^T) = det(A)?", check_transpose_invariance(A))
    dD, D = det_triangular([2,3,5]); print("Triangular det (should be 2*3*5):", dD)

## 5) Determinant via LU decomposition (with permutations)

For `PA = LU` with permutation matrix `P`, we have `det(A) = det(P) * det(L) * det(U)`; since `det(L)=1` for unit-lower, this becomes
`det(A) = sgn(P) * (product of diag(U))`.

In [ ]:
def det_via_LU(A):
    A = to_M(A)
    L, U, perm = A.LUdecomposition()
    # SymPy returns a permutation list 'perm' such that row swaps were applied.
    # Compute sign from number of transpositions in the permutation.
    # 'perm' gives where each row moved; estimate parity by comparing to identity.
    sign = 1
    visited = [False]*len(perm)
    for i in range(len(perm)):
        if not visited[i]:
            cycle_len = 0
            j = i
            while not visited[j]:
                visited[j] = True
                j = perm[j]
                cycle_len += 1
            if cycle_len > 0:
                sign *= (-1)**(cycle_len-1)
    detU = sp.prod(U[i,i] for i in range(U.shape[0]))
    return sp.simplify(sign * detU), L, U, perm

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[1,2,1],[0,1,3],[2,0,1]])
    dLU, L, U, p = det_via_LU(A)
    print("det via LU:", dLU, " | SymPy det:", A.det(), " | perm:", p)

## 6) Cramer's rule

If `det(A) ≠ 0`, the solution of `A x = b` has components `x_i = det(A_i)/det(A)`, where `A_i` is `A` with column `i` replaced by `b`.

In [ ]:
def cramers_rule(A, b):
    A = to_M(A); b = to_M(b)
    assert A.shape[0] == A.shape[1] == b.shape[0], "Shapes must be compatible and A square."
    dA = A.det()
    if dA == 0:
        raise ValueError("Cramer's rule requires det(A) != 0.")
    n = A.shape[0]
    x = sp.zeros(n, 1)
    for i in range(n):
        Ai = A.copy()
        Ai[:, i] = b
        x[i, 0] = Ai.det() / dA
    return x

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[2, -1, 0],[ -1, 2, -1],[0, -1, 2]])
    b = sp.Matrix([1, 0, 1])
    x_cramer = cramers_rule(A, b)
    x_sym = A.LUsolve(b)
    print("Cramer's == LUsolve ? ", x_cramer.equals(x_sym))

## 7) Geometric meaning: area/volume = |det|

- In 2D, the area of the parallelogram spanned by columns of `A` equals `|det(A)|`.
- In 3D, the volume of the parallelepiped equals `|det(A)|`.
Below we compute the values and show simple plots.

In [ ]:
def area_2d(A):
    A = np.asarray(A, dtype=float)
    assert A.shape == (2,2)
    return abs(np.linalg.det(A))

def volume_3d(A):
    A = np.asarray(A, dtype=float)
    assert A.shape == (3,3)
    return abs(np.linalg.det(A))

def plot_parallelogram(A):
    A = np.asarray(A, dtype=float)
    assert A.shape == (2,2)
    e1 = np.array([0,0]); a = A[:,0]; b = A[:,1]
    verts = np.array([e1, a, a+b, b, e1])
    plt.figure()
    plt.plot(verts[:,0], verts[:,1], marker='o')
    plt.axis('equal')
    plt.title("Parallelogram spanned by columns of A")

def plot_parallelepiped(A):
    A = np.asarray(A, dtype=float)
    assert A.shape == (3,3)
    a, b, c = A[:,0], A[:,1], A[:,2]
    base = np.array([[0,0,0], a, a+b, b, [0,0,0]])
    top  = base + c
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(base[:,0], base[:,1], base[:,2])
    ax.plot(top[:,0], top[:,1], top[:,2])
    # vertical edges
    for p in base:
        q = p + c
        ax.plot([p[0], q[0]], [p[1], q[1]], [p[2], q[2]])
    ax.set_title("Parallelepiped spanned by columns of A")

# Demo
if __name__ == "__main__":
    A2 = np.array([[2,1],[1,3]], float)
    A3 = np.array([[1,0,2],[0,2,1],[1,1,1]], float)
    print("Area 2D |det|:", area_2d(A2))
    print("Vol 3D  |det|:", volume_3d(A3))
    # Uncomment to visualize in Colab:
    # plot_parallelogram(A2)
    # plot_parallelepiped(A3)

## 8) Determinant and eigenvalues

For a square matrix over \(\mathbb{C}\), the determinant equals the product of eigenvalues (counted with algebraic multiplicity).
We also check that the constant term of the characteristic polynomial equals \((-1)^n \det(A)\).

In [ ]:
def eigenvalues_product_equals_det(A):
    A = to_M(A)
    eigs = list(A.eigenvals().keys())  # distinct eigenvalues (SymPy dict), multiplicity in values
    # Reconstruct product with multiplicities
    prod = sp.Integer(1)
    for val, mult in A.eigenvals().items():
        prod *= val**mult
    return sp.simplify(prod), A.det()

def charpoly_constant_det_relation(A):
    A = to_M(A)
    x = sp.symbols('x')
    p = A.charpoly(x).as_expr()
    n = A.shape[0]
    const_term = sp.expand(p.subs(x, 0))
    return sp.simplify(const_term), (-1)**n * A.det()

# Demo
if __name__ == "__main__":
    A = sp.Matrix([[0,1,0],[0,0,1],[-1,-1,-1]])
    prod, d = eigenvalues_product_equals_det(A)
    print("∏ eigenvalues == det(A)? ", sp.simplify(prod - d) == 0)
    c0, target = charpoly_constant_det_relation(A)
    print("charpoly const term == (-1)^n det(A)? ", sp.simplify(c0 - target) == 0)

---

### How to use this notebook

1. Run the **Setup** cell.
2. Compare **`det_leibniz`** with `Matrix.det()` on small matrices.
3. Use **`laplace_expand_row/col`** to see cofactor expansion steps.
4. Compute **adjugate/inverse** via cofactors using **`adjugate`** / **`inverse_via_adjugate`**.
5. Explore **row-operation effects** and multiplicativity.
6. Get det via **LU** using **`det_via_LU`**; compare with `A.det()`.
7. Solve small systems via **Cramer's rule** (exact arithmetic).
8. Visualize **area/volume** and check eigenvalue products.